# **LangChain `create_agent`**

---

## **1. 설치 및 환경 설정**

```bash
pip install langchain langchain-openai langchain-tavily langgraph
```



In [1]:
from dotenv import load_dotenv
load_dotenv()

True

## **2. 기본 사용법**

In [2]:
from langchain.agents import create_agent

# 기본 에이전트 생성 (도구 없음)
agent = create_agent(
    model="openai:gpt-4.1-nano",
    tools=[],
    system_prompt="You are a helpful assistant."
)

# 에이전트 실행
result = agent.invoke({
    "messages": [
        {"role": "user", "content": "안녕하세요!"}
    ]
})

print(result["messages"][-1].content)

안녕하세요! 어떻게 도와드릴까요?


## **3. 도구(Tools) 사용**

`(1) 기본 Tavily 검색`

In [3]:
from langchain.agents import create_agent
from langchain_tavily import TavilySearch

# Tavily 검색 도구 초기화
search_tool = TavilySearch(
    max_results=5,                    # 최대 검색 결과 수
    topic="general",                  # 검색 주제
)

# 검색 도구를 포함한 에이전트 생성
agent = create_agent(
    model="openai:gpt-4.1-nano",
    tools=[search_tool],
    system_prompt="You are a helpful research assistant that can search the web."
)

# 에이전트 실행
result = agent.invoke({
    "messages": [
        {"role": "user", "content": "2024년 노벨 물리학상 수상자는 누구인가요?"}
    ]
})

print(result["messages"][-1].content)

2024년 노벨 물리학상 수상자는 존 J. 홉필드(John J. Hopfield)와 제프리 힌튼(Geoffrey Hinton)입니다. 홉필드는 신경망의 물리적 모델인 '홉필드 네트워크'를 발명했고, 힌튼은 이 네트워크를 기반으로 한 머신러닝 연구로 인공지능 분야의 발전에 기여한 공로로 수상하였다고 합니다.


`(2) 동적 파라미터 설정`

- 에이전트가 검색 시 동적으로 파라미터를 설정 가능

In [4]:
from langchain.agents import create_agent
from langchain_tavily import TavilySearch

search_tool = TavilySearch(
    max_results=5,
)

agent = create_agent(
    model="openai:gpt-4.1-mini",
    tools=[search_tool],
    system_prompt="""You are a research assistant. 
    When searching for academic content, use include_domains=['wikipedia.org'].
    For news, use topic='news'."""
)

In [5]:
# 에이전트가 자동으로 적절한 검색 파라미터를 선택 (Wikipedia 검색)
result = agent.invoke({
    "messages": [
        {"role": "user", 
         "content": "Find information about quantum computing from Wikipedia only."}
    ]
})

for msg in result["messages"]:
    msg.pretty_print()

================================ Human Message =================================

Find information about quantum computing from Wikipedia only.
================================== Ai Message ==================================
Tool Calls:
  tavily_search (call_DNe44kpeiHSLucN1Upl64DM3)
 Call ID: call_DNe44kpeiHSLucN1Upl64DM3
  Args:
    query: Quantum computing
    include_domains: ['wikipedia.org']
    search_depth: advanced
================================= Tool Message =================================
Name: tavily_search

{"query": "Quantum computing", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://simple.wikipedia.org/wiki/Quantum_computer", "title": "Quantum computer - Simple English Wikipedia, the free ...", "content": "A quantum computer is a model of how to build a computer. The idea is that quantum computers can use certain ideas from quantum mechanics, such as superposition and entanglement, to perform operations on data. The basic prin

In [6]:
# 에이전트가 동적으로 파라미터 설정 (최신 뉴스 검색)
result = agent.invoke({
    "messages": [
        {"role": "user", 
         "content": "Get me the latest news about artificial intelligence from the past week."}
    ]
})

for msg in result["messages"]:
    msg.pretty_print()

================================ Human Message =================================

Get me the latest news about artificial intelligence from the past week.
================================== Ai Message ==================================
Tool Calls:
  tavily_search (call_cS4oZ0BCR20pyQ2uXdhqePD3)
 Call ID: call_cS4oZ0BCR20pyQ2uXdhqePD3
  Args:
    query: artificial intelligence
    time_range: week
    topic: news
================================= Tool Message =================================
Name: tavily_search

{"query": "artificial intelligence", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://forklog.com/en/zoom-ceo-predicts-ai-will-reduce-workweek-to-3-4-days/", "title": "Zoom CEO Predicts AI Will Reduce Workweek to 3-4 Days - ForkLog", "score": 0.47753036, "published_date": "Tue, 28 Oct 2025 11:47:38 GMT", "content": "AI translated#Artificial Intelligence#Layoffs#Opinions # Zoom CEO Predicts AI Will Reduce Workweek to 3-4 Days Artificial int

`(3) 순차적 도구 사용`

In [7]:
from langchain_tavily import TavilySearch, TavilyExtract

# 검색 도구
search_tool = TavilySearch(
    max_results=5,
    topic="general"
)

# 콘텐츠 추출 도구
extract_tool = TavilyExtract(
    extract_depth="basic", 
    include_images=False
)

agent = create_agent(
    model="openai:gpt-4.1-mini",
    tools=[search_tool, extract_tool],
    system_prompt="""You are a research assistant.
    - Use tavily_search to find relevant URLs
    - Use tavily_extract to get detailed content from specific URLs
    """
)

result = agent.invoke({
    "messages": [
        {"role": "user", 
         "content": "최신 AI 연구 동향을 찾아서 상세 내용을 추출해주세요."}
    ]
})

for msg in result["messages"]:
    msg.pretty_print()

================================ Human Message =================================

최신 AI 연구 동향을 찾아서 상세 내용을 추출해주세요.
================================== Ai Message ==================================
Tool Calls:
  tavily_search (call_yKS1B68DDFTHRQTNZcaebMAk)
 Call ID: call_yKS1B68DDFTHRQTNZcaebMAk
  Args:
    query: 최신 AI 연구 동향
    search_depth: advanced
    include_images: False
    time_range: month
    topic: general
================================= Tool Message =================================
Name: tavily_search

{"query": "최신 AI 연구 동향", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://blog.naver.com/PostView.naver?blogId=trizdoctor&logNo=224048210103", "title": "AI 혁신과 투자 붐, 메타·오픈AI·앤트로픽의 신기술 및 규제 변화", "content": "| 최신AI기술동향  TechCrunch 2025-10-21 | AI 혁신과 투자 붐, 메타·오픈AI·앤트로픽의 신기술 및 규제 변화  트리즈씽킹   ・  2025. 10. 21. 7:06  URL 복사  이웃추가 본문 기타 기능 공유하기  신고하기  오늘날 인공지능(AI)은 우리의 일상뿐만 아니라 과학, 의료, 소프트웨어 개발 등 다양한 산업 분야를 근본적으로 재편하고 있습니다. 최첨단 연구자들이 주도하는 혁신적

## **4. Middleware 설정**

`(1) 대화 요약 Middleware`

In [8]:
from langchain.agents import create_agent
from langchain.agents.middleware import SummarizationMiddleware
from langgraph.checkpoint.memory import InMemorySaver

agent = create_agent(
    model="openai:gpt-4.1-mini",
    tools=[search_tool],
    middleware=[
        SummarizationMiddleware(
            model="openai:gpt-4.1-nano",     # 요약에 사용할 모델
            max_tokens_before_summary=500,   # 요약 시작 임계값 (토큰 수)
            messages_to_keep=3,              # 요약 후에도 유지할 메시지 수
        )
    ],
    checkpointer=InMemorySaver(),
)

In [9]:
result = agent.invoke({
    "messages": [
        {"role": "user", "content": "최신 AI 연구 동향을 찾아서 설명해주세요."}
    ]
},
config={"configurable": {"thread_id": "custom_thread_001"}})

for msg in result["messages"]:
    msg.pretty_print()

================================ Human Message =================================

최신 AI 연구 동향을 찾아서 설명해주세요.
================================== Ai Message ==================================
Tool Calls:
  tavily_search (call_8knmarlZ7amx9d2LDOppMEFW)
 Call ID: call_8knmarlZ7amx9d2LDOppMEFW
  Args:
    query: 최신 AI 연구 동향
    search_depth: advanced
    time_range: month
    topic: general
================================= Tool Message =================================
Name: tavily_search

{"query": "최신 AI 연구 동향", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://blog.naver.com/PostView.naver?blogId=trizdoctor&logNo=224048210103", "title": "AI 혁신과 투자 붐, 메타·오픈AI·앤트로픽의 신기술 및 규제 변화", "content": "| 최신AI기술동향  TechCrunch 2025-10-21 | AI 혁신과 투자 붐, 메타·오픈AI·앤트로픽의 신기술 및 규제 변화  트리즈씽킹   ・  2025. 10. 21. 7:06  URL 복사  이웃추가 본문 기타 기능 공유하기  신고하기  오늘날 인공지능(AI)은 우리의 일상뿐만 아니라 과학, 의료, 소프트웨어 개발 등 다양한 산업 분야를 근본적으로 재편하고 있습니다. 최첨단 연구자들이 주도하는 혁신적 스타트업부터, AI 기반 영상 플랫폼과 코딩 도구의 급성장

In [10]:
# 대화 계속
result = agent.invoke({
    "messages": [
        {"role": "user", "content": "LLM과 생성형 AI를 중심으로 자세하게 조사해주세요."}
    ]
},
config={"configurable": {"thread_id": "custom_thread_001"}})

for msg in result["messages"]:
    msg.pretty_print()

================================ Human Message =================================

Here is a summary of the conversation to date:

- 최근 한 달간의 AI 연구 및 산업 동향 요약:
  - AI 투자와 혁신: OpenAI, 메타, 앤트로픽 등 글로벌 기업과 스타트업이 AI 기술에 대규모 투자. AI는 과학, 의료, 소프트웨어 등 다양한 산업에서 혁신 주도.
  - 규제와 기술 발전: 캘리포니아 AI 안전법 등 규제가 AI 발전과 상생하는 사례. 강화 학습은 명확 평가 기준이 있는 분야에서 빠르게 발전, 주관적 평가 분야는 더딤.
  - 국제 협력 및 연구: 국내외 AI 석학들이 참여하는 심포지엄에서 AI 거버넌스, 미래 기술, 사회적 영향 논의. 차세대 AI 연구성과와 글로벌 협력 강화.
  - 산업별 AI 적용과 한계: AI는 특정 문제 집중 학습 중이며, 데이터 품질과 레이블링이 핵심 과제. 제로샷 능력 등 한계 존재.
  - 전반적으로 AI는 안전성과 혁신의 균형을 이루며, 미래 신뢰받는 AI 개발과 글로벌 협력에 집중하고 있음.
================================ Human Message =================================

LLM과 생성형 AI를 중심으로 자세하게 조사해주세요.
================================== Ai Message ==================================
Tool Calls:
  tavily_search (call_NFO1OegdJliDyj9I0aOkRaDr)
 Call ID: call_NFO1OegdJliDyj9I0aOkRaDr
  Args:
    query: LLM과 생성형 AI 최신 연구 동향
    search_depth: advanced
    time_range: month
    topic: general
==========

In [11]:
# 대화 계속
result = agent.invoke({
    "messages": [
        {"role": "user", "content": "에너지 부족 문제에 대해 조사해주세요."}
    ]
},
config={"configurable": {"thread_id": "custom_thread_001"}})

for msg in result["messages"]:
    msg.pretty_print()

================================ Human Message =================================

Here is a summary of the conversation to date:

최근 한 달간 LLM(대규모 언어 모델)과 생성형 AI 연구 및 산업 동향은 다음과 같습니다:
- 모델 중심 혁신: 데이터 중심에서 전환, 학습 방식·구조·시스템 통합 재정의, 자가학습·자율 개선 시스템 등장, 파라미터 효율적 기술(LoRA, Adapter) 활용.
- 생성형 AI & 멀티모달 융합: ‘이해하는 AI’에서 ‘창조하는 AI’로 발전, 멀티모달 융합, 에이전트형 AI·자율성 증대, 예술·산업 적용 확대.
- 산업·국가 경쟁: 글로벌·국내 기업의 대규모 투자·개발, 네이버 ‘하이파클로바X’ 오픈소스화, 카카오 오픈AI 협력 강화, AI 윤리·안전성 기술 발전.
- 비판·미래 방향: 얀 르쿤 교수 등은 현재 LLM의 한계와 5년 내 중요성 감소 예측, ‘월드 모델’ 등 인간 수준 AI 개발에 집중 필요.
================================ Human Message =================================

에너지 부족 문제에 대해 조사해주세요.
================================== Ai Message ==================================
Tool Calls:
  tavily_search (call_kpjwHUozk0xd3U1LZwC2XmBG)
 Call ID: call_kpjwHUozk0xd3U1LZwC2XmBG
  Args:
    query: 에너지 부족 문제 최신 동향
    search_depth: advanced
    time_range: month
    topic: general
================================= Tool Message ===============================

`(2) 커스텀 Middleware 생성`

In [12]:
from langchain.agents.middleware import AgentMiddleware
from typing import Any, Dict

class LoggingMiddleware(AgentMiddleware):
    """모든 도구 호출을 로깅하는 Middleware"""
    
    def before_model(self, state: Dict[str, Any], runtime) -> Dict[str, Any] | None:
        """모델 호출 전에 실행"""
        print(f"🤖 모델 호출 전: {len(state['messages'])} 메시지")
        return None
    
    def after_model(self, state: Dict[str, Any], runtime) -> Dict[str, Any] | None:
        """모델 호출 후에 실행"""
        last_message = state["messages"][-1]
        if hasattr(last_message, "tool_calls") and last_message.tool_calls:
            print(f"🔧 도구 호출: {[tc['name'] for tc in last_message.tool_calls]}")
        return None

class RateLimitMiddleware(AgentMiddleware):
    """도구 호출 속도 제한"""
    
    def __init__(self, max_calls_per_minute: int = 10):
        self.max_calls = max_calls_per_minute
        self.call_times = []
    
    def wrap_tool_call(self, request, handler):
        """도구 호출을 래핑하여 속도 제한 적용"""
        import time
        from datetime import datetime, timedelta
        
        # 최근 1분 내 호출 정리
        now = datetime.now()
        self.call_times = [t for t in self.call_times 
                          if now - t < timedelta(minutes=1)]
        
        # 속도 제한 확인
        if len(self.call_times) >= self.max_calls:
            raise Exception(f"속도 제한: 분당 최대 {self.max_calls}회 호출 가능")
        
        # 도구 실행
        self.call_times.append(now)
        return handler(request)

agent = create_agent(
    model="openai:gpt-4.1-nano",
    tools=[search_tool],
    middleware=[
        LoggingMiddleware(),
        RateLimitMiddleware(max_calls_per_minute=1) # 테스트용으로 1분에 1회 제한
    ]
)


In [13]:
# 에이전트 실행 (로깅 확인)
result = agent.invoke({
    "messages": [{"role": "user", "content": "2024년 노벨 물리학상 수상자는 누구인가요?"}]
})

for msg in result["messages"]:
    msg.pretty_print()

🤖 모델 호출 전: 1 메시지
🔧 도구 호출: ['tavily_search']
🤖 모델 호출 전: 3 메시지
================================ Human Message =================================

2024년 노벨 물리학상 수상자는 누구인가요?
================================== Ai Message ==================================
Tool Calls:
  tavily_search (call_lopXUiW3MUK7Z7s4cRdMDlji)
 Call ID: call_lopXUiW3MUK7Z7s4cRdMDlji
  Args:
    query: 2024년 노벨 물리학상 수상자
    search_depth: advanced
================================= Tool Message =================================
Name: tavily_search

{"query": "2024년 노벨 물리학상 수상자", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://www.newskorea.ne.kr/news/articleView.html?idxno=12736", "title": "2024년 노벨 물리학상 수상자 - 뉴스코리아(NEWS KOREA)", "content": "6개 시·군 21개 기업, 4,448억 원 투자 유치\n\n자활사업으로 일구는 ‘내일’의 희망\n\n㈜몬스터에프앤에스, 천안지역 취약계층 위해 1200만원 상당 후원\n\n## 본문영역\n\n### 2024년 노벨 물리학상 수상자\n\n#### 노벨 물리학상\n\n 주강호 특파원 newsjebo@newskorea.ne.kr\n 입력 2024.10.11 12:07\n\n 글씨크기\n\n이 기사를 공유합니다\n\n(뉴스코리아=스톡홀름) 주강호

In [21]:
# 의도된 오류 발생.
result = agent.invoke({
    "messages": [{"role": "user", "content": "2020년대 노벨 물리학상 수상자는 누구인가요?"}]
})

for msg in result["messages"]:
    msg.pretty_print()

ValueError: Checkpointer requires one or more of the following 'configurable' keys: thread_id, checkpoint_ns, checkpoint_id

`(3) 대화 영속성 (Checkpointing)`

- Checkpointing을 통해 대화의 상태를 저장하고 복원 가능

In [15]:
from langgraph.checkpoint.memory import InMemorySaver
from langchain.agents import create_agent

# 메모리 체크포인터 생성
checkpointer = InMemorySaver()

agent = create_agent(
    model="openai:gpt-4.1-nano",
    tools=[search_tool],
    checkpointer=checkpointer
)

# 첫 번째 대화
config1 = {"configurable": {"thread_id": "user-123-session-1"}}
result1 = agent.invoke(
    {"messages": [{"role": "user", "content": "내 이름은 김철수야"}]},
    config1
)
for msg in result1["messages"]:
    msg.pretty_print()

================================ Human Message =================================

내 이름은 김철수야
================================== Ai Message ==================================

반갑습니다, 김철수님! 무엇을 도와드릴까요?


In [16]:
# 두 번째 대화 (같은 thread_id로 이전 대화 기억)
result2 = agent.invoke(
    {"messages": [{"role": "user", "content": "내 이름이 뭐였지?"}]},
    config1
)
for msg in result2["messages"]:
    msg.pretty_print()  # "김철수"라고 기억함

================================ Human Message =================================

내 이름은 김철수야
================================== Ai Message ==================================

반갑습니다, 김철수님! 무엇을 도와드릴까요?
================================ Human Message =================================

내 이름이 뭐였지?
================================== Ai Message ==================================

당신의 이름은 김철수라고 말씀하셨습니다.


In [17]:
# 다른 대화 (다른 thread_id로 새로운 세션)
config2 = {"configurable": {"thread_id": "user-123-session-2"}}
result3 = agent.invoke(
    {"messages": [{"role": "user", "content": "내 이름이 뭐야?"}]},
    config2
)
for msg in result3["messages"]:
    msg.pretty_print()  # 이전 대화 기억 없음

================================ Human Message =================================

내 이름이 뭐야?
================================== Ai Message ==================================

죄송하지만, 저는 당신의 이름을 알지 못합니다. 혹시 원하시면 알려주실 수 있나요?


## **5. 실전 프로젝트**

`(1) 웹 리서치 에이전트`

In [18]:
from langchain.agents import create_agent
from langchain_openai import ChatOpenAI
from langchain_tavily import TavilySearch, TavilyExtract
from langgraph.checkpoint.memory import InMemorySaver

# 도구 설정
search_tool = TavilySearch(
    max_results=5,
    topic="general",
    search_depth="advanced"
)

extract_tool = TavilyExtract(
    extract_depth="advanced"
)

# 모델 인스턴스를 직접 생성하여 reasoning 설정
model = ChatOpenAI(
    model="gpt-4.1-nano",  # reasoning 지원 모델
    temperature=0.3,  
    max_completion_tokens=5000
)

# 에이전트 생성
research_agent = create_agent(
    model=model,  # 모델 인스턴스 전달
    tools=[search_tool, extract_tool],   # 도구 등록
    system_prompt="""You are an expert research assistant.

When conducting research:
1. Use tavily_search to find relevant sources
2. Evaluate the credibility of sources
3. Use tavily_extract to get detailed information from the most relevant URLs
4. Synthesize information from multiple sources
5. Provide citations for all claims

Format your response as:
## Research Summary
[Brief overview]

## Key Findings
- Finding 1 [Source]
- Finding 2 [Source]

## Detailed Analysis
[In-depth analysis]

## Sources
- [URL 1]
- [URL 2]
""",
    checkpointer=InMemorySaver()
)

# 실행
result = research_agent.invoke({
    "messages": [
        {"role": "user", 
         "content": "2024년 AI 산업의 주요 트렌드를 조사하고 분석해주세요."}
    ]
}, {"configurable": {"thread_id": "research-1"}})

print(result["messages"][-1].content)


## Research Summary
2024년 AI 산업은 다양한 기술적 발전과 트렌드가 두드러지고 있습니다. 주요 내용은 AI의 전반적 확산, 생성형 AI의 성장, AI의 경제적 기여, 그리고 산업별 응용 확대 등입니다. 특히, 대형 언어 모델(LLM), Retrieval-Augmented Generation(RAG), AI 기반의 디지털 트윈, 3D 시각화, AI 안전성 및 신뢰성 강화 등이 중요한 트렌드로 부상하고 있습니다. 또한, 글로벌 기업들이 AI를 활용한 혁신과 경쟁력 확보에 집중하며, AI 규제와 윤리적 문제 해결도 중요한 이슈로 대두되고 있습니다.

## Key Findings
- AI의 전반적 확산과 산업별 응용 확대가 2024년 주요 트렌드로 자리잡고 있으며, 특히 LLM과 RAG 기술이 핵심 역할을 하고 있음 [KITA, 2024].
- 생성형 AI 시장은 2024년부터 2034년까지 연평균 성장률이 24.3%에 달하며, 글로벌 시장 규모는 수백억 달러에 이를 전망 [GMI Insights, 2024].
- AI의 경제적 기여는 2024년부터 2034년까지 연평균 24.3% 성장하며, 글로벌 GDP에 미치는 영향도 확대될 것으로 기대됨 [GMI Insights, 2024].
- AI 안전성과 신뢰성 확보를 위한 연구와 규제 강화가 지속되며, AI의 윤리적 문제와 투명성 제고가 중요한 과제로 부상하고 있음 [NVIDIA, 2024].

## Detailed Analysis
2024년 AI 산업은 이미 확산된 기술들이 산업 전반에 깊숙이 자리잡으며, 다양한 응용 분야에서 혁신을 이끌고 있습니다. 특히, 대형 언어 모델(LLM)은 자연어 처리, 고객 서비스, 콘텐츠 생성 등에서 핵심 역할을 하며, Retrieval-Augmented Generation(RAG) 기술은 정보 검색과 생성의 결합으로 더욱 정밀한 AI 서비스를 가능하게 하고 있습니다. 이러한 기술들은 금융, 의료, 제조, 미디어 등 다양한 산업에서 활용도가 높아지고 있으며, AI 기

`(2) SQL 데이터베이스 에이전트`

In [19]:
from langchain.agents import create_agent
from langchain_openai import ChatOpenAI
from langchain.tools import tool
from langchain_community.utilities import SQLDatabase

# 데이터베이스 연결
db = SQLDatabase.from_uri("sqlite:///etf_database.db")

@tool
def list_tables() -> str:
    """데이터베이스의 모든 테이블을 나열합니다."""
    tables = db.get_table_names()
    return f"사용 가능한 테이블: {', '.join(tables)}"

@tool
def get_schema(table_name: str) -> str:
    """특정 테이블의 스키마를 가져옵니다."""
    return db.get_table_info([table_name])

@tool
def run_query(query: str) -> str:
    """SQL 쿼리를 실행합니다. SELECT 쿼리만 허용됩니다."""
    try:
        if not query.strip().upper().startswith('SELECT'):
            return "Error: Only SELECT queries are allowed"
        result = db.run(query)
        return str(result)
    except Exception as e:
        return f"Error: {str(e)}"

model = ChatOpenAI(
    model="gpt-4.1-mini",
    temperature=0,
    max_completion_tokens=2000
)

sql_agent = create_agent(
    model=model,
    tools=[list_tables, get_schema, run_query],
    system_prompt="""You are a SQL expert assistant for ETF database queries.

Available tables:
- ETFs: Contains ETF information with columns like 종목코드, 종목명, 수익률_최근1년, 순자산총액, etc.
- ETFsInfo: Contains detailed ETF information

Instructions:
1. Check the schema ONCE using get_schema("ETFs")
2. Write ONE SQL query to answer the question
3. Execute the query using run_query
4. Present the result to the user
5. STOP - Do NOT call any more tools after presenting the result

Important:
- After executing run_query and getting results, immediately provide the answer to the user
- Do NOT repeat tool calls
- One schema check + one query execution = DONE

Safety guidelines:
- Only use SELECT queries
- Never modify data
- Validate column names match the schema"""
)

# 실행
result = sql_agent.invoke(
    {"messages": [{"role": "user", "content": "수익률이 가장 높은 ETF는 무엇인가요?"}]},
    config={"recursion_limit": 15}
)

for msg in result["messages"]:
    msg.pretty_print()


================================ Human Message =================================

수익률이 가장 높은 ETF는 무엇인가요?
================================== Ai Message ==================================
Tool Calls:
  get_schema (call_wCFRovfLl7XHqyFF9DgzLRPP)
 Call ID: call_wCFRovfLl7XHqyFF9DgzLRPP
  Args:
    table_name: ETFs
================================= Tool Message =================================
Name: get_schema


CREATE TABLE "ETFs" (
	"종목코드" TEXT, 
	"종목명" TEXT, 
	"상장일" TEXT, 
	"분류체계" TEXT, 
	"운용사" TEXT, 
	"수익률_최근1년" REAL, 
	"기초지수" TEXT, 
	"추적오차" REAL, 
	"순자산총액" REAL, 
	"괴리율" REAL, 
	"변동성" TEXT, 
	"복제방법" TEXT, 
	"총보수" REAL, 
	"과세유형" TEXT, 
	PRIMARY KEY ("종목코드")
)

/*
3 rows from ETFs table:
종목코드	종목명	상장일	분류체계	운용사	수익률_최근1년	기초지수	추적오차	순자산총액	괴리율	변동성	복제방법	총보수	과세유형
466400	1Q 25-08 회사채(A+이상)액티브	2023/09/19	채권-회사채-단기	하나자산운용	4.52	KIS 2025-08만기형 크레딧 A+이상 지수(총수익)	0.11	111916276404.0	0.03	매우낮음	실물(액티브)	0.1	배당소득세(보유기간과세)
491610	1Q CD금리액티브(합성)	2024/09/24	기타	하나자산운용	0.0	KIS 하나 CD금리 총수익지수	0.05	316206006696.0	0

In [20]:
# 실행
result = sql_agent.invoke(
    {"messages": [{"role": "user", "content": "순자산총액이 가장 큰 ETF 5개를 알려주세요."}]},
    config={"recursion_limit": 15}
)

for msg in result["messages"]:
    msg.pretty_print()


================================ Human Message =================================

순자산총액이 가장 큰 ETF 5개를 알려주세요.
================================== Ai Message ==================================
Tool Calls:
  get_schema (call_HY7ltvY73b1TbgOtPj8s5XY7)
 Call ID: call_HY7ltvY73b1TbgOtPj8s5XY7
  Args:
    table_name: ETFs
================================= Tool Message =================================
Name: get_schema


CREATE TABLE "ETFs" (
	"종목코드" TEXT, 
	"종목명" TEXT, 
	"상장일" TEXT, 
	"분류체계" TEXT, 
	"운용사" TEXT, 
	"수익률_최근1년" REAL, 
	"기초지수" TEXT, 
	"추적오차" REAL, 
	"순자산총액" REAL, 
	"괴리율" REAL, 
	"변동성" TEXT, 
	"복제방법" TEXT, 
	"총보수" REAL, 
	"과세유형" TEXT, 
	PRIMARY KEY ("종목코드")
)

/*
3 rows from ETFs table:
종목코드	종목명	상장일	분류체계	운용사	수익률_최근1년	기초지수	추적오차	순자산총액	괴리율	변동성	복제방법	총보수	과세유형
466400	1Q 25-08 회사채(A+이상)액티브	2023/09/19	채권-회사채-단기	하나자산운용	4.52	KIS 2025-08만기형 크레딧 A+이상 지수(총수익)	0.11	111916276404.0	0.03	매우낮음	실물(액티브)	0.1	배당소득세(보유기간과세)
491610	1Q CD금리액티브(합성)	2024/09/24	기타	하나자산운용	0.0	KIS 하나 CD금리 총수익지수	0.05	316206006696